In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21908599
paper_name = 'minear_cyert_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 42 x 7


In [7]:
original_data.head()

,Rank,Gene,Locus,Defect vs WT,Retested,Retested sensitive,Description
0,1,YPK1,YKL126W,6.61,-,NaN,76.5 kDa Serine/threonine protein kinase with ...
1,2,AFT1,YGL071W,5.07,+,+,Involved in iron homeostasis and affects cell ...
2,3,YDR269C,YDR269C,4.27,+,+,Dubious ORF; deletion prevents expression of C...
3,4,FET3,YMR058W,4.24,+,+,Ferro-O2-oxidoreductase required for high-affi...
4,5,PHO86,YJL117W,4.18,-,NaN,Protein specifically required for packaging of...


In [8]:
original_data['Locus'] = original_data['Locus'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['Locus'] = clean_orf(original_data['Locus'])

In [10]:
# Translate to ORFs 
original_data['Locus'] = translate_sc(original_data['Locus'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['Locus'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Rank, Gene, Locus, Defect vs WT, Retested, Retested sensitive, Description]
Index: []


In [12]:
original_data.set_index('Locus', inplace=True)
original_data.index.name='orf'

In [13]:
original_data['data'] = -original_data['Defect vs WT']

In [14]:
original_data = original_data[['data']].copy()

In [15]:
for c in original_data.columns:
    original_data[c] = pd.to_numeric(original_data[c], errors='coerce')

# Prepare the final dataset

In [16]:
data = original_data.copy()

In [17]:
dataset_ids = [16484]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,16484
data_type,value
orf,
YKL126W,-6.61
YGL071W,-5.07
YDR269C,-4.27
YMR058W,-4.24
YJL117W,-4.18


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16484
,data_type,value
gene_id,orf,
1524,YKL126W,-6.61
2752,YGL071W,-5.07
2396,YDR269C,-4.27
4351,YMR058W,-4.24
3358,YJL117W,-4.18


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16484           
data_type       value     valuez
gene_id orf                     
1524    YKL126W -6.61 -19.816912
2752    YGL071W -5.07 -15.199961
2396    YDR269C -4.27 -12.801545
4351    YMR058W -4.24 -12.711604
3358    YJL117W -4.18 -12.531723

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 21908599...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  6.82it/s]


Updating the data_modified_on field...
